**Troubleshooting Scripts - Transaction Log**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Getting list of heap tables**

  

Evaluate if they would benefit from clustered indexes (see index usage scripts below)

In [ ]:
SELECT 
    t.object_id
    ,s.name + '.' + t.name AS [table]
    ,p.rows
FROM 
    sys.tables t WITH (NOLOCK) 
        JOIN sys.schemas s WITH (NOLOCK) ON
            t.schema_id = s.schema_id
        CROSS APPLY
         (
            SELECT SUM(p.rows) AS [rows]
            FROM sys.partitions p WITH (NOLOCK) 
            WHERE t.object_id = p.object_id AND p.index_id = 0
        ) p
WHERE
    t.is_memory_optimized = 0 AND -- SQL Server 2014+
    t.is_ms_shipped = 0 AND
    EXISTS 
     (
        SELECT *
        FROM sys.indexes i WITH (NOLOCK)
        WHERE t.object_id = i.object_id AND i.index_id = 0
    )
ORDER BY
    p.rows DESC
OPTION (RECOMPILE, MAXDOP 1);

**Inefficient heap tables**

  

Look at number of forwarded pointers as they add an overhead during data access. Also review internal fragmentation as it wastes space Rebuild tables as needed.

In [ ]:
SELECT TOP 25
    t.object_id
    ,s.name + '.' + t.name AS [table]
    ,SUM(ips.record_count) AS [rows]
    ,SUM(ips.forwarded_record_count) 
            AS [forwarding pointers]
    ,SUM(ips.avg_page_space_used_in_percent * ips.page_count) / 
        NULLIF(SUM(ips.page_count),0) 
            AS [internal fragmentation %]
FROM 
    sys.tables t WITH (NOLOCK) 
        JOIN sys.schemas s WITH (NOLOCK) ON
            t.schema_id = s.schema_id
        CROSS APPLY
            sys.dm_db_index_physical_stats
                (DB_ID(),t.object_id,0,NULL,'DETAILED') ips
WHERE
    t.is_memory_optimized = 0 AND -- SQL Server 2014+
    t.is_ms_shipped = 0 AND
    EXISTS 
    (
        SELECT *
        FROM sys.indexes i WITH (NOLOCK)
        WHERE t.object_id = i.object_id AND i.index_id = 0
    )
GROUP BY
    t.object_id, s.name, t.name
ORDER BY
     [forwarding pointers] DESC 
OPTION (RECOMPILE, MAXDOP 1);

**Uniqueidentifiers**

  

Usually. they introduce fragmentation and other issues.

In [ ]:
SELECT
    t.object_id
    ,s.name + '.' + t.name AS [table]
    ,i.name AS [index]
    ,i.is_disabled
    ,p.rows
FROM 
    sys.tables t WITH (NOLOCK) 
        JOIN sys.schemas s WITH (NOLOCK) ON
            t.schema_id = s.schema_id
        JOIN sys.indexes i WITH (NOLOCK) ON
            t.object_id = i.object_id
        CROSS APPLY
        (
            SELECT SUM(p.rows) AS [rows]
            FROM sys.partitions p WITH (NOLOCK) 
            WHERE i.object_id = p.object_id AND i.index_id = p.index_id
        ) p
WHERE
    t.is_memory_optimized = 0 AND -- SQL Server 2014+
    i.type in (1,2) AND /* CI and NCI */
    i.is_hypothetical = 0 AND
    EXISTS 
    (
        SELECT *
        FROM 
            sys.index_columns ic WITH (NOLOCK)
                JOIN sys.columns c WITH (NOLOCK) ON
                    ic.object_id = c.object_id AND
                    ic.column_id = c.column_id
        WHERE 
            ic.object_id = i.object_id AND
            ic.index_id = i.index_id AND
            ic.key_ordinal = 1 AND
            c.system_type_id = 36 /* uniqueidentifier */
    )
ORDER BY
    p.[rows] DESC
OPTION (RECOMPILE, MAXDOP 1);

**Wide clustered indexes**

In [ ]:
SELECT TOP 25
    t.object_id
    ,s.name + '.' + t.name AS [table]
    ,p.rows
    ,ic.[max length]
FROM 
    sys.tables t WITH (NOLOCK) 
        JOIN sys.schemas s WITH (NOLOCK) ON
            t.schema_id = s.schema_id
        CROSS APPLY
        (
            SELECT SUM(p.rows) AS [rows]
            FROM sys.partitions p WITH (NOLOCK) 
            WHERE t.object_id = p.object_id AND p.index_id = 1
        ) p
        CROSS APPLY
        (
            SELECT SUM(c.max_length) as [max length]
            FROM 
                sys.indexes i  
                    JOIN sys.index_columns ic WITH (NOLOCK) ON
                        i.object_id = ic.object_id AND
                        i.index_id = ic.index_id AND
                        ic.is_included_column = 0
                    JOIN sys.columns c WITH (NOLOCK) ON
                        ic.object_id = c.object_id AND
                        ic.column_id = c.column_id
            WHERE 
                i.object_id = t.object_id AND
                i.index_id = 1 AND
                i.type = 1
        ) ic
WHERE
    t.is_memory_optimized = 0 AND -- SQL Server 2014+
    1 = 1
ORDER BY
    ic.[max length] DESC
OPTION (RECOMPILE, MAXDOP 1);

**Non-unique clustered indexes**

  

Check if data there is unique and redefine them if possible

In [ ]:
SELECT 
    t.object_id
    ,s.name + '.' + t.name AS [table]
    ,p.rows
FROM 
    sys.tables t WITH (NOLOCK) 
        JOIN sys.schemas s WITH (NOLOCK) ON
            t.schema_id = s.schema_id
        CROSS APPLY
        (
            SELECT SUM(p.rows) AS [rows]
            FROM sys.partitions p WITH (NOLOCK) 
            WHERE t.object_id = p.object_id AND p.index_id = 1
        ) p
WHERE
    t.is_memory_optimized = 0 AND -- SQL Server 2014+
    EXISTS 
    (
        SELECT *
        FROM sys.indexes i WITH (NOLOCK)
        WHERE 
            t.object_id = i.object_id AND
            i.index_id = 1 AND
            i.is_unique = 0 AND
            i.type = 1 /* CI */
    )
ORDER BY
    p.[rows] DESC
OPTION (RECOMPILE, MAXDOP 1);

**Untrusted foreign keys**

  

Validate foreign keys with ALTER TABLE WITH CHECK CHECK CONSTRAINT statement. This will lock the table with Sch-M lock for the duration of validation

In [ ]:
SELECT
    fk.is_disabled
    ,fk.is_not_trusted
    ,fk.name AS [FK]
    ,ps.name + '.' + pt.name AS [Referencing Table / Detail]
    ,rs.name + '.' + rt.name AS [Referenced Table / Master]
    ,fk.update_referential_action_desc
    ,fk.delete_referential_action_desc
FROM 
    sys.foreign_keys fk WITH (NOLOCK) 
        JOIN sys.tables pt WITH (NOLOCK) ON 
            fk.parent_object_id = pt.object_id 
        JOIN sys.schemas ps WITH (NOLOCK) ON 
            pt.schema_id = ps.schema_id
        JOIN sys.tables rt WITH (NOLOCK) ON 
            fk.referenced_object_id = rt.object_id 
        JOIN sys.schemas rs WITH (NOLOCK) ON 
            rt.schema_id = rs.schema_id
WHERE
    fk.is_not_trusted = 1 OR fk.is_disabled = 1
OPTION (RECOMPILE, MAXDOP 1);

**Potentially non-indexed foreign keys**

  

Review the output and create indexes as needed. The script will return false positive if there are filtered indexes that support referential integrity

In [ ]:
SELECT
    fk.is_disabled
    ,fk.is_not_trusted
    ,fk.name as [FK]
    ,ps.name + '.' + pt.name AS [Referencing Table / Detail]
    ,rs.name + '.' + rt.name AS [Referenced Table / Master]
    ,fk.update_referential_action_desc
    ,fk.delete_referential_action_desc
    ,fk_cols.cols as [fk columns]
FROM 
    sys.foreign_keys fk WITH (NOLOCK) 
        JOIN sys.tables pt WITH (NOLOCK) ON 
            fk.parent_object_id = pt.object_id 
        JOIN sys.schemas ps WITH (NOLOCK) ON 
            pt.schema_id = ps.schema_id
        JOIN sys.tables rt WITH (NOLOCK) ON 
            fk.referenced_object_id = rt.object_id 
        JOIN sys.schemas rs WITH (NOLOCK) ON 
            rt.schema_id = rs.schema_id
        CROSS APPLY
        (
            SELECT 
                (
                    SELECT 
                        UPPER(col.name) AS [text()]
                        ,',' AS [text()]
                    FROM 
                        sys.foreign_key_columns fkc WITH (NOLOCK)
                            JOIN sys.columns col WITH (NOLOCK) ON
                                fkc.parent_object_id = col.object_id AND
                                fkc.parent_column_id = col.column_id
                    WHERE 
                        fkc.constraint_object_id = fk.object_id
                    ORDER BY 
                        fkc.constraint_column_id
                    FOR XML PATH('')
                ) as cols
        ) fk_cols
WHERE
    NOT EXISTS
    (
        SELECT *
        FROM 
            sys.indexes i WITH (NOLOCK)
                CROSS APPLY
                (
                    SELECT
                        (
                            SELECT
                                UPPER(col.name) AS [text()]
                                ,',' AS [text()]
                            FROM 
                                sys.index_columns ic WITH (NOLOCK) 
                                    JOIN sys.columns col WITH (NOLOCK) ON
                                        ic.object_id = col.object_id AND
                                        ic.column_id = col.column_id
                            WHERE
                                i.object_id = ic.object_id AND
                                i.index_id = ic.index_id AND
                                ic.is_included_column = 0
                            ORDER BY
                                ic.partition_ordinal
                            FOR XML PATH('')
                        ) AS cols
                ) idx_col
        WHERE
            i.object_id = fk.parent_object_id AND
            CHARINDEX(fk_cols.cols,idx_col.cols) = 1 AND
            i.is_disabled = 0 AND
            i.is_hypothetical = 0 AND
            i.has_filter = 0 AND
            i.type IN (1,2)
    )
ORDER BY
    [Referenced Table / Master]
OPTION (RECOMPILE, MAXDOP 1);

**Potentially redundant indexes**

  

Review the indexes in the output especially those with FullOverlap. Use for index consolidation

In [ ]:
SELECT
    s.name + '.' + t.name AS [Table]
    ,i1.index_id AS [I1 ID]
    ,i1.name AS [I1 Name]
    ,dupIdx.index_id AS [I2 ID]
    ,dupIdx.name AS [I2 Name] 
    ,LEFT(i1_col.key_col,LEN(i1_col.key_col) - 1) AS [I1 Keys]
    ,LEFT(i1_col.included_col,LEN(i1_col.included_col) - 1) AS [I1 Included Col]
    ,i1.filter_definition AS [I1 Filter]
    ,LEFT(i2_col.key_col,LEN(i2_col.key_col) - 1) AS [I2 Keys]
    ,LEFT(i2_col.included_col,LEN(i2_col.included_col) - 1) AS [I2 Included Col]
    ,dupIdx.filter_definition AS [I2 Filter]
    ,IIF(
        CHARINDEX(i1_col.key_col, i2_col.key_col) = 1 OR
        CHARINDEX(i2_col.key_col, i1_col.key_col) = 1,'Yes','No'
    ) AS [Fully Redundant]
FROM 
    sys.tables t WITH (NOLOCK) 
        JOIN sys.indexes i1 wITH (NOLOCK) ON
            t.object_id = i1.object_id
        JOIN sys.index_columns ic1 WITH (NOLOCK) ON
            ic1.object_id = i1.object_id AND
            ic1.index_id = i1.index_id AND 
            ic1.key_ordinal = 1  
        JOIN sys.columns c WITH (NOLOCK) ON
            c.object_id = ic1.object_id AND
            c.column_id = ic1.column_id      
        JOIN sys.schemas s WITH (NOLOCK) ON 
            t.schema_id = s.schema_id
        CROSS APPLY
        (
            SELECT i2.index_id, i2.name, i2.filter_definition
            FROM
                sys.indexes i2 WITH (NOLOCK) 
                    JOIN sys.index_columns ic2 WITH (NOLOCK) ON       
                        ic2.object_id = i2.object_id AND
                        ic2.index_id = i2.index_id AND 
                        ic2.key_ordinal = 1  
            WHERE    
                i2.object_id = i1.object_id AND 
                i2.index_id > i1.index_id AND 
                ic2.column_id = ic1.column_id AND
                i2.type in (1,2) AND
                i2.is_disabled = 0 AND 
                i2.is_hypothetical = 0 AND
                (
                    i1.has_filter = i2.has_filter AND
                    ISNULL(i1.filter_definition,'') = 
                        ISNULL(i2.filter_definition,'')
                )
        ) dupIdx
        CROSS APPLY
        (
            SELECT
                (
                    SELECT 
                        col.name AS [text()]
                        ,IIF(icol_meta.is_descending_key = 1, ' DESC','') 
                            AS [text()]
                        ,',' AS [text()]
                    FROM                 
                        sys.index_columns icol_meta WITH (NOLOCK) 
                            JOIN sys.columns col WITH (NOLOCK) ON
                                icol_meta.object_id = col.object_id AND
                                icol_meta.column_id = col.column_id
                    WHERE
                        icol_meta.object_id = i1.object_id AND
                        icol_meta.index_id = i1.index_id AND
                        icol_meta.is_included_column = 0
                    ORDER BY
                        icol_meta.key_ordinal
                    FOR XML PATH('')
                ) AS key_col
                ,(                
                    SELECT 
                        col.name AS [text()]
                        ,',' AS [text()]
                    FROM                 
                        sys.index_columns icol_meta WITH (NOLOCK) 
                            JOIN sys.columns col WITH (NOLOCK) ON
                                icol_meta.object_id = col.object_id AND
                                icol_meta.column_id = col.column_id
                    WHERE
                        icol_meta.object_id = i1.object_id AND
                        icol_meta.index_id = i1.index_id AND
                        icol_meta.is_included_column = 1
                    ORDER BY
                        col.name
                    FOR XML PATH('')
                ) AS included_col
        ) i1_col
        CROSS APPLY
        (
            SELECT
                (
                    SELECT 
                        col.name AS [text()]
                        ,IIF(icol_meta.is_descending_key = 1, ' DESC','') 
                            AS [text()]
                        ,',' AS [text()]
                    FROM                 
                        sys.index_columns icol_meta WITH (NOLOCK) 
                            JOIN sys.columns col WITH (NOLOCK) ON
                                icol_meta.object_id = col.object_id AND
                                icol_meta.column_id = col.column_id
                    WHERE
                        icol_meta.object_id = t.object_id AND
                        icol_meta.index_id = dupIdx.index_id AND
                        icol_meta.is_included_column = 0
                    ORDER BY
                        icol_meta.key_ordinal
                    FOR XML PATH('')
                ) AS key_col
                ,(                
                    SELECT 
                        col.name AS [text()]
                        ,',' AS [text()]
                    FROM                 
                        sys.index_columns icol_meta WITH (NOLOCK) 
                            JOIN sys.columns col WITH (NOLOCK) ON
                                icol_meta.object_id = col.object_id AND
                                icol_meta.column_id = col.column_id
                    WHERE
                        icol_meta.object_id = t.object_id AND
                        icol_meta.index_id = dupIdx.index_id AND
                        icol_meta.is_included_column = 1
                    ORDER BY
                        col.name
                    FOR XML PATH('')
                ) AS included_col
        ) i2_col
WHERE
    i1.is_disabled = 0 AND
    i1.is_hypothetical = 0 AND
    i1.type in (1,2) 
ORDER BY
    s.name, t.name, i1.index_id
OPTION (RECOMPILE, MAXDOP 1);


**High identity values**

In [ ]:
DECLARE
	@Types TABLE
	(
		type_id INT NOT NULL PRIMARY KEY,
		name SYSNAME NOT NULL,
		max_val DECIMAL(38) NOT NULL
	)

INSERT INTO @Types(type_id, name, max_val)
VALUES
	(48,'TINYINT',255)
	,(52,'SMALLINT',32767)
	,(56,'INT',2147483647)
	,(127,'BIGINT',9223372036854775807)
	,(108,'NUMERIC',99999999999999999999999999999999999999)  -- 10^38-1
	,(106,'DECIMAL',99999999999999999999999999999999999999); -- 10^38-1

DECLARE 
	@percentThreshold INT = 50;

;WITH CTE
AS
(
	SELECT 
		s.name + '.' + t.name AS [table]
		,c.name AS [column]
		,tp.name + IIF(tp.type_id IN (106,108), '(' + CONVERT(VARCHAR(2),c.precision) + ')','') AS [type]
		,CONVERT(DECIMAL(38),IDENT_CURRENT(t.name)) AS [identity]
		,CASE
			WHEN tp.type_id IN (106,108) 
			THEN 
				CASE 
					WHEN c.precision < 38
					THEN POWER(CONVERT(DECIMAL(38),10),c.precision) - 1
					ELSE tp.max_val
				END
			ELSE
				tp.max_val
		END AS [max value]
	FROM
		sys.tables t WITH (NOLOCK) 
			JOIN sys.schemas s WITH (NOLOCK) ON
				t.schema_id = s.schema_id
			JOIN sys.columns c WITH (NOLOCK) ON 
				c.object_id = t.object_id
			JOIN @Types tp ON
				tp.type_id = c.system_type_id
	WHERE  
		c.is_identity = 1 
)
SELECT *,CONVERT(DECIMAL(6,3),[identity] / [max value] * 100.) AS [percent full] 
FROM CTE
WHERE CONVERT(DECIMAL(6,3),[identity] / [max value] * 100.) > @percentThreshold 
ORDER BY [percent full] DESC;

**High sequence values**

In [ ]:
DECLARE
	@Types TABLE
	(
		type_id INT NOT NULL PRIMARY KEY,
		name SYSNAME NOT NULL,
		max_val DECIMAL(38) NOT NULL
	)

INSERT INTO @Types(type_id, name, max_val)
VALUES
	(48,'TINYINT',255)
	,(52,'SMALLINT',32767)
	,(56,'INT',2147483647)
	,(127,'BIGINT',9223372036854775807)
	,(108,'NUMERIC',99999999999999999999999999999999999999)  -- 10^38-1
	,(106,'DECIMAL',99999999999999999999999999999999999999); -- 10^38-1

DECLARE 
	@percentThreshold INT = 50;

;WITH CTE
AS
(
	SELECT 
		s.name + '.' + seq.name AS [sequence]
		,tp.name AS [type]
		,CASE tp.type_id
			WHEN 48 THEN CONVERT(DECIMAL(38),CONVERT(TINYINT,seq.current_value))
			WHEN 52 THEN CONVERT(DECIMAL(38),CONVERT(SMALLINT,seq.current_value))
			WHEN 56 THEN CONVERT(DECIMAL(38),CONVERT(INT,seq.current_value))
			WHEN 127 THEN CONVERT(DECIMAL(38),CONVERT(BIGINT,seq.current_value))
			WHEN 106 THEN CONVERT(DECIMAL(38),seq.current_value)
			WHEN 108 THEN CONVERT(DECIMAL(38),seq.current_value)
		END as [current]
		,CASE tp.type_id
			WHEN 48 THEN CONVERT(DECIMAL(38),CONVERT(TINYINT,seq.maximum_value))
			WHEN 52 THEN CONVERT(DECIMAL(38),CONVERT(SMALLINT,seq.maximum_value))
			WHEN 56 THEN CONVERT(DECIMAL(38),CONVERT(INT,seq.maximum_value))
			WHEN 127 THEN CONVERT(DECIMAL(38),CONVERT(BIGINT,seq.maximum_value))
			WHEN 106 THEN CONVERT(DECIMAL(38),seq.maximum_value)
			WHEN 108 THEN CONVERT(DECIMAL(38),seq.maximum_value)
		END AS [max value]	
	FROM
		sys.sequences seq WITH (NOLOCK) 
			JOIN sys.schemas s WITH (NOLOCK) ON
				seq.schema_id = s.schema_id
			JOIN @Types tp ON
				tp.type_id = seq.system_type_id
)
SELECT *, CONVERT(DECIMAL(6,3), [current] / [max value] * 100.) as [percent full] 
FROM CTE
WHERE CONVERT(DECIMAL(6,3), [current] / [max value] * 100.) > @percentThreshold 
ORDER BY [percent full] DESC;

**Maximum values of index keys**

Credits to Erland Sommarskog ([https://www.sommarskog.se](https://www.sommarskog.se))

In [ ]:
DECLARE
    @Types table
    (
        type_id int not null primary key,
        name sysname not null,
        max_val bigint not null
    )

INSERT INTO @Types(type_id, name, max_val)
VALUES
    (48,'tinyint',255)
    ,(52,'smallint',32767)
    ,(56,'int',2147483647)
    ,(127,'bigint',9223372036854775807
)

DECLARE @sql nvarchar(MAX),
        @tblname nvarchar(512),
        @colname nvarchar(256),
        @maxkey bigint,
        @maxval bigint,
        @threshold decimal(10,2) = 0.5,
        @tblcur CURSOR

SET @tblcur = CURSOR STATIC FOR
   SELECT quotename(s.name) + '.' +  quotename(t.name), quotename(c.name),
          Ty.max_val
   FROM   sys.tables t
   JOIN   sys.schemas s ON s.schema_id = t.schema_id
   JOIN   sys.columns c ON t.object_id = c.object_id
   JOIN   @Types Ty ON c.system_type_id = Ty.type_id
   WHERE  EXISTS (SELECT *
                  FROM   sys.indexes i
                  JOIN   sys.index_columns ic ON ic.object_id = i.object_id
                                             AND ic.index_id  = i.index_id
                                             AND ic.key_ordinal = 1
                  WHERE  i.object_id = t.object_id
                    AND  ic.column_id = c.column_id
                    AND  (i.is_primary_key = 1 OR
                          i.is_unique_constraint = 1)
                    AND  i.is_hypothetical = 0
                    AND  NOT EXISTS (SELECT *
                                     FROM   sys.index_columns ic2
                                     WHERE  ic2.object_id = i.object_id
                                       AND  ic2.index_id  = i.index_id
                                       AND  ic2.key_ordinal > 1))
   ORDER BY 1
                      

OPEN @tblcur

WHILE 1 = 1
BEGIN 
   FETCH @tblcur INTO @tblname, @colname, @maxval
   IF @@fetch_status <> 0 
      BREAK

   SELECT @sql = 'SELECT @maxval = MAX(' + @colname + ') FROM ' + @tblname
   PRINT @sql
   EXEC sp_executesql @sql, N'@maxval bigint OUTPUT', @maxkey OUTPUT

   IF @maxkey > @threshold * @maxval
      SELECT @tblname, @colname, @maxkey
END


**Index usage (sys.dm\_db\_index\_usage\_stats)**

Simple version. Use sp\_IndexAnalysis scripts for holistic view

In [ ]:
SELECT 
    t.object_id
    ,i.index_id
    ,s.name + '.' + t.name AS [Table]
    ,i.name AS [Index] 
    ,i.type_desc
    ,i.has_filter AS [Filtered]
    ,i.is_unique AS [Unique]
    ,p.rows AS [Rows]
    ,ius.user_seeks AS [Seeks]
    ,ius.user_scans AS [Scans]
    ,ius.user_lookups AS [Lookups]
    ,ius.user_seeks + ius.user_scans + ius.user_lookups AS [Reads]
    ,ius.user_updates AS [Updates]
    ,ius.last_user_seek AS [Last Seek]
    ,ius.last_user_scan AS [Last Scan]
    ,ius.last_user_lookup AS [Last Lookup]
    ,ius.last_user_update AS [Last Update]
FROM 
    sys.tables t WITH (NOLOCK) 
        JOIN sys.indexes i WITH (NOLOCK) ON
            t.object_id = i.object_id
        JOIN sys.schemas s WITH (NOLOCK) ON 
            t.schema_id = s.schema_id
        CROSS APPLY
        (
            SELECT SUM(p.rows) AS [rows]
            FROM sys.partitions p WITH (NOLOCK) 
            WHERE 
                i.object_id = p.object_id AND
                i.index_id = p.index_id
        ) p
        LEFT OUTER JOIN sys.dm_db_index_usage_stats ius ON
            ius.database_id = DB_ID() AND
            ius.object_id = i.object_id AND
            ius.index_id = i.index_id
WHERE
    t.is_memory_optimized = 0 AND -- SQL Server 2014+
    i.is_disabled = 0 AND 
    i.is_hypothetical = 0 AND
    t.is_ms_shipped = 0
ORDER BY
    s.name, t.name, i.index_id
OPTION (RECOMPILE, MAXDOP 1);

**Index usage based on plan cache data**

May take some time to execute in the systems with large number of plans cached.

Set proper index name in the script

In [ ]:
DECLARE
    @IndexName SYSNAME = QUOTENAME('<INDEX NAME>');

;WITH XMLNAMESPACES
(DEFAULT 'http://schemas.microsoft.com/sqlserver/2004/07/showplan')  
,CachedData
AS
(
    SELECT DISTINCT
        obj.value('@Database','SYSNAME') AS [Database]
        ,obj.value('@Schema','SYSNAME') + '.' + obj.value('@Table','SYSNAME') 
            AS [Table]
        ,obj.value('@Index','SYSNAME') AS [Index]
        ,obj.value('@IndexKind','VARCHAR(64)') AS [Type]
        ,stmt.value('@StatementText', 'NVARCHAR(MAX)') AS [Statement]
        ,CONVERT(NVARCHAR(MAX),qp.query_plan) AS query_plan
        ,cp.plan_handle
    FROM
        sys.dm_exec_cached_plans cp WITH (NOLOCK) 
            CROSS APPLY sys.dm_exec_query_plan(plan_handle) qp
            CROSS APPLY query_plan.nodes
             ('/ShowPlanXML/BatchSequence/Batch/Statements/StmtSimple') batch(stmt)
            CROSS APPLY stmt.nodes
             ('.//IndexScan/Object[@Index=sql:variable("@IndexName")]') idx(obj)
)
SELECT
    cd.[Database]
    ,cd.[Table]
    ,cd.[Index]
    ,cd.[Type]
    ,cd.[Statement]
    ,CONVERT(XML,cd.query_plan) AS query_plan
    ,qs.execution_count
    ,(qs.total_logical_reads + qs.total_logical_writes) / qs.execution_count 
        AS [Avg IO]
    ,qs.total_logical_reads
    ,qs.total_logical_writes
    ,qs.total_worker_time
    ,qs.total_worker_time / qs.execution_count / 1000 AS [Avg Worker Time (ms)]
    ,qs.total_rows
    ,qs.creation_time
    ,qs.last_execution_time
FROM 
    CachedData cd
        OUTER APPLY
         (
            SELECT 
                SUM(qs.execution_count) AS execution_count
                ,SUM(qs.total_logical_reads) AS total_logical_reads 
                ,SUM(qs.total_logical_writes) AS total_logical_writes
                ,SUM(qs.total_worker_time) AS total_worker_time
                ,SUM(qs.total_rows) AS total_rows
                ,MIN(qs.creation_time) AS creation_time 
                ,MAX(qs.last_execution_time) AS last_execution_time
            FROM sys.dm_exec_query_stats qs WITH (NOLOCK)
            WHERE qs.plan_handle = cd.plan_handle
        ) qs
OPTION (RECOMPILE, MAXDOP 1);

**Index Operational Statistics (sys.dm\_db\_index\_operational\_stats)**

Simple version. Use sp\_IndexAnalysis scripts for holistic view

In [ ]:
SELECT 
	t.object_id
	,i.index_id
	,s.name + '.' + t.name AS [Table]
	,i.name AS [Index] 
	,i.type_desc
	,i.has_filter AS [Filtered]
	,i.is_unique AS [Unique]
	,p.rows AS [Rows]
	,ous.*
FROM 
	sys.tables t WITH (NOLOCK) 
		JOIN sys.indexes i WITH (NOLOCK) ON
			t.object_id = i.object_id
		JOIN sys.schemas s WITH (NOLOCK) ON 
			t.schema_id = s.schema_id
		CROSS APPLY
		(
			SELECT SUM(p.rows) AS [rows]
			FROM sys.partitions p WITH (NOLOCK) 
			WHERE 
				i.object_id = p.object_id AND
				i.index_id = p.index_id
		) p
		OUTER APPLY sys.dm_db_index_operational_stats
			(DB_ID(),i.object_id,i.index_id,NULL) ous
WHERE
    t.is_memory_optimized = 0 AND -- SQL Server 2014+
	i.is_disabled = 0 AND 
	i.is_hypothetical = 0 AND
	t.is_ms_shipped = 0
ORDER BY
	s.name, t.name, i.index_id
OPTION (RECOMPILE, MAXDOP 1);